In [11]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import defaultdict

In [2]:
df1 = pd.read_csv("./chr1.csv", index_col="strain")
df2 = pd.read_csv("./chr2.csv", index_col="strain")
df1 = df1.join(df2, how="left")

In [16]:
df1

,accession1,accession2
strain,,
10329_clinical,CP045794.1,CP045795.1
16-VB00198,CP097355.1,CP097356.1
160807,CP033141.1,CP033142.1
17-VB00214,CP062153.1,CP062154.1
19-021-D1_ahpnd,CP046411.1,CP046412.1
...,...,...
vp-HL-202005,CP114194.1,CP114195.1
vp-HL-202006,CP150902.1,CP150903.1
vp-HL-202008,CP150909.1,CP150910.1


In [3]:
accession_to_strain = dict(zip(df1['accession1'], df1.index))
accession_to_strain

In [49]:
delta_1 = []
accession_to_strain = dict(zip(df1['accession1'], df1.index)) 

# First pass: calculate delta_1
with open("./MAFFT_chr1.fasta", "r") as infile:
    for line in infile:
        if line.startswith('>'):
            coordinate_chr1 = line[1:].split(":")[1]
            start = coordinate_chr1.split("-")[0]
            end = coordinate_chr1.split("-")[1]
            delta_1.append(int(end) - int(start))

# Define expected coordinate based on median delta
expected_coordinate = [500, 500 + int(max(delta_1))]

# # Second pass: extract sequence within expected_coordinate
with open("./MAFFT_chr1.fasta", "r") as infile, open("./MAFFT_chr1_renamed.fasta", "w") as outfile:
    sequence = ""
    header = ""
    for line in infile:
        line = line.strip()
        if line.startswith('>'):
            # Process previous record if it exists
            if header and sequence:
                subseq = sequence[expected_coordinate[0]:expected_coordinate[1]]
                outfile.write(f"{header}\n")
                for i in range(0, len(subseq), 60):
                    outfile.write(subseq[i:i+60] + "\n")
            # Process new header
            accession = line[1:].split(':')[0]
            strain_name = accession_to_strain.get(accession, accession)
            strain_clean = strain_name
            header = f">{strain_clean}"
            sequence = ""  # Reset sequence

        else:
            sequence += line

    # Process the last record
    if header and sequence:
        subseq = sequence[expected_coordinate[0]:expected_coordinate[1]]
        outfile.write(f"{header}\n")
        for i in range(0, len(subseq), 60):
            outfile.write(subseq[i:i+60] + "\n")

In [39]:
np.average(delta_1)

37816.984375

In [75]:
expected_coordinate

[500, 37836]

In [53]:
delta_2 = []
accession_to_strain = dict(zip(df1['accession2'], df1.index))  # Ensure this is defined

# First pass: calculate delta_1
with open("./MAFFT_chr2.fasta", "r") as infile:
    for line in infile:
        if line.startswith('>'):
            coordinate_chr1 = line[1:].split(":")[1]
            start = coordinate_chr1.split("-")[0]
            end = coordinate_chr1.split("-")[1]
            delta_2.append(int(end) - int(start))

# Define expected coordinate based on median delta
expected_coordinate = [1, 1 + int(max(delta_2))]

# Second pass: extract sequence within expected_coordinate
with open("./MAFFT_chr2.fasta", "r") as infile, open("./MAFFT_chr2_renamed.fasta", "w") as outfile:
    sequence = ""
    header = ""
    for line in infile:
        line = line.strip()
        if line.startswith('>'):
            # Process previous record if it exists
            if header and sequence:
                subseq = sequence[expected_coordinate[0]:expected_coordinate[1]]
                outfile.write(f"{header}\n")
                for i in range(0, len(subseq), 60):
                    outfile.write(subseq[i:i+60] + "\n")

            # Process new header
            accession = line[1:].split(':')[0]
            strain_name = accession_to_strain.get(accession, accession)
            strain_clean = strain_name
            header = f">{strain_clean}"

            sequence = ""  # Reset sequence

        else:
            sequence += line

    # Process the last record
    if header and sequence:
        subseq = sequence[expected_coordinate[0]:expected_coordinate[1]]
        outfile.write(f"{header}\n")
        for i in range(0, len(subseq), 60):
            outfile.write(subseq[i:i+60] + "\n")

In [56]:
max(delta_2)

1984

In [7]:
expected_coordinate

[1, 1984]

In [54]:
def read_fasta(filename):
    seq_dict = {}
    with open(filename, 'r') as f:
        current_label = None
        seq_list = []
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if current_label:
                    seq_dict[current_label] = ''.join(seq_list)
                current_label = line[1:]  # remove '>'
                seq_list = []
            else:
                seq_list.append(line)
        if current_label:
            seq_dict[current_label] = ''.join(seq_list)
    return seq_dict

In [55]:
seq1 = read_fasta("./MAFFT_chr1_renamed.fasta")
seq2 = read_fasta("./MAFFT_chr2_renamed.fasta")
output = "combined_chr12.fasta"

# Combine sequences by strain name
combined = defaultdict(str)

for strain in seq1:
    combined[strain] += seq1.get(strain, "")

for strain in seq2:
    combined[strain] += seq2.get(strain, "")

# Write combined sequences to output FASTA
with open(output, 'w') as f:
    for strain, sequence in combined.items():
        f.write(f">{strain}\n")
        # Write sequence with line length 60 (optional for readability)
        for i in range(0, len(sequence), 60):
            f.write(sequence[i:i+60] + "\n")

### Rename formatted from treelife file

In [ ]:
newer = {}
for strain, plasmid in strain_name.items():
    strain = strain.split("_ahpnd")[0]
    newer[strain] = plasmid
new = {}
for strain, plasmid in newer.items():
    strain = strain.split("_clinical")[0]
    new[strain] = plasmid

In [ ]:
import re
with open("./concat_ready.nex.treefile", "r") as f:
    tree_string = f.read()
def replace_label(match):
    label = match.group(0)
    prefix = label.split("_")[0]
    return plasmid_to_strain.get(prefix, label)
new_tree = re.sub(r'([A-Z]{2}\d+\.\d+_\d+-\d+)', replace_label, tree_string)
with open("./T6SS2_auto_hi.nhx", "w") as f:
    f.write(new_tree)